In [1]:
! pip list | grep langchain
! pip list | grep opensearch

langchain                     0.0.330
opensearch-py                 2.3.2


In [2]:
import sys, os
module_path = ".."
sys.path.append(os.path.abspath(module_path))

In [3]:
import json
import boto3
from pprint import pprint 
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

os.environ["AWS_DEFAULT_REGION"] = "us-east-1"

boto3_bedrock_cli = bedrock.get_bedrock_client(
  assumed_role= os.environ.get("BEDROCK_ASSUME_ROLE", None),
  endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
  region=os.environ.get("AWS_DEFAULT_REGION", None)
)

Create new client
  Using region: us-east-1
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


3. Audi A8 펑크 타이어 수리 질문

In [5]:
prompt_data = """당신은 도움이 되는 조수입니다. 간결한 방식으로 질문에 답하시오 당신이 확실하지 않은 경우
대답은 '확실하지 않습니다.'라고 말하세요.

질문: Audo A8에서 펑크 난 타이어를 어떻게 고칠 수 있습니까?
답변:"""

body = json.dumps({
  "prompt": f"\n\nHuman:{prompt_data}\n\nAssistant:",
  "max_tokens_to_sample": 500,
  "temperature": 0.3,
  "top_k": 250,
  "top_p": 1,
  "stop_sequences": ["\n\nHuman:"]
})

In [9]:
modelId = 'anthropic.claude-v2'
accept = 'application/json'
contentType = 'application/json'

def inference_model(body, modelId, accept, contentType):
  response = boto3_bedrock_cli.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
  )
  response_body = json.loads(response.get("body").read())
  answer = response_body.get("completion")

  print_ww("prompt_data: \n", prompt_data)
  print("### Answer from Claude-v2 ###")
  print_ww(answer.strip())

inference_model(body, modelId, accept, contentType)

prompt_data:
 당신은 도움이 되는 조수입니다. 간결한 방식으로 질문에 답하시오 당신이 확실하지 않은 경우
대답은 '확실하지 않습니다.'라고 말하세요.

질문: Audo A8에서 펑크 난 타이어를 어떻게 고칠 수 있습니까?
답변:
### Answer from Claude-v2 ###
확실하지 않습니다. Audo A8에 대한 제 경험이 제한적이기 때문에 펑크난 타이어를 고치는 정확한 방법을 알려드리기 어렵습니다. 가장 좋은 방법은 Audo 서비스 센터에 연락하여
전문가의 도움을 받는 것이 좋겠습니다.


In [10]:
prompt_data = "Amazon Tirana에서 펑크 난 타이어를 어떻게 고칠 수 있습니까?"
body = json.dumps({
    "prompt": f"\n\nHuman:{prompt_data}\n\nAssistant:",
    "max_tokens_to_sample": 500,
    "temperature": 0.3,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["\n\nHuman:"]            
    }) 

inference_model(body, modelId, accept, contentType)    

prompt_data:
 Amazon Tirana에서 펑크 난 타이어를 어떻게 고칠 수 있습니까?
### Answer from Claude-v2 ###
제가 아는 한 알바니아의 수도 티라나에서 펑크난 타이어를 고치는 몇 가지 방법이 있습니다:

- 타이어 수리점 방문 - 티라나에는 많은 타이어 수리점이 있습니다. 그들은 펑크난 타이어를 수리하고 교체할 수 있습니다. 수리 비용은 타이어의 손상 정도에 따라 다릅니다.

- 도로 원조 서비스 이용 - 알바니아의 고속도로들에는 도로 원조 서비스가 있습니다. 그들은 현장에서 펑크난 타이어를 수리하거나 교체해줍니다. 이 서비스를 이용하려면 일반적으로
수수료를 지불해야 합니다.

- 스페어 타이어 사용 - 차에 스페어 타이어가 있다면, 펑크난 타이어를 스페어 타이어로 교체할 수 있습니다. 그러나 스페어 타이어는 일시적인 해결책이므로 가능한 빨리 정규
타이어를 수리하거나 교체하는 것이 좋습니다.

- 모바일 타이어 서비스 이용 - 티라나에는 차량으로 고객을 찾아가 펑크난 타이어를 현장에서 수리하거나 교체해주는 모바일 타이어 서비스도 있습니다.

가장 좋은 방법은 전문 타이어 수리점을 방문하는


5. 환각을 억제하기 위한 Context 제공

In [11]:
context = """타이어 및 타이어 공기압:

타이어는 검은색 고무로 만들어졌으며 차량의 바퀴에 장착됩니다. 주행, 코너링 및 제동에 필요한 그립을 제공합니다. 타이어 공기압과 타이어 마모는 차량의 성능과 핸들링에 영향을 미칠 수 있으므로 고려해야 할 두 가지 중요한 요소입니다.

권장 타이어 공기압을 찾을 수 있는 곳:

차량의 운전석 쪽 B-필러에 있는 공기 주입 라벨에서 권장 타이어 공기압 사양을 확인할 수 있습니다. 또는 차량 설명서에서 이 정보를 참조할 수 있습니다. 권장 타이어 공기압은 속도와 탑승자 수 또는 차량의 최대 적재량에 따라 달라질 수 있습니다.

타이어 재공기:

타이어 공기압을 점검할 때는 타이어가 차가울 때 확인하는 것이 중요합니다. 이는 타이어가 주변 온도와 동일한 온도가 되도록 차량을 최소 3시간 동안 그대로 두는 것을 의미합니다.

타이어를 다시 채우려면:

    차량의 권장 타이어 공기압을 확인하십시오.
    공기 펌프에 제공된 지침을 따르고 타이어에 올바른 공기압을 주입하십시오.
    차량의 중앙 화면에서 "차량 상태" 앱을 엽니다.
    "타이어 공기압" 탭으로 이동합니다.
    "압력 보정" 옵션을 누르고 조치를 확인하십시오.
    타이어 공기압을 보정하기 위해 30km/h 이상의 속도로 몇 분 동안 차를 운전하십시오.

참고: 경우에 따라 타이어 공기압과 관련된 경고 기호 또는 메시지를 지우기 위해 15분 이상 운전해야 할 수도 있습니다. 경고가 지속되면 타이어를 식히고 위의 단계를 반복하십시오.

바람 빠진 타이어:

운전 중 타이어 펑크가 났을 경우 일시적으로 펑크를 봉인하고 타이어 이동성 키트를 사용하여 타이어에 다시 공기를 주입할 수 있습니다. 이 키트는 일반적으로 차량의 수하물 보관함 안감 아래에 보관됩니다.

타이어 이동성 키트 사용 지침:

    차량의 테일게이트 또는 트렁크를 엽니다.
    타이어 모빌리티 키트에 접근하려면 러기지 영역의 라이닝을 들어 올리십시오.
    타이어 이동성 키트와 함께 제공된 지침에 따라 타이어의 펑크를 밀봉하십시오.
    키트를 사용한 후에는 원래 위치에 안전하게 다시 넣어야 합니다.
    사용한 실란트 병의 폐기 및 교체에 대한 지원은 Rivesla 또는 적절한 서비스에 문의하십시오.

타이어 모빌리티 키트는 임시 솔루션이며 최대 10분 또는 8km(먼저 도래하는 것) 동안 최대 80km/h의 속도로 운전할 수 있도록 설계되었습니다. 펑크 난 타이어는 가능한 한 빨리 교체하거나 전문가에게 수리를 맡기는 것이 좋습니다.
"""

6. Context를 제공하여 Claude-v2에 질문

In [12]:
question = "Audi A8의 펑크 난 타이어를 어떻게 고칠 수 있습니까?"
prompt_data = f"""## 사이에 제공된 정보만을 기준으로 질문에 답하고 단계별 안내를 제공합니다.
#
{context}
#

Question: {question}
"""

In [13]:
print("prompt_data: \n", prompt_data)

prompt_data: 
 ## 사이에 제공된 정보만을 기준으로 질문에 답하고 단계별 안내를 제공합니다.
#
타이어 및 타이어 공기압:

타이어는 검은색 고무로 만들어졌으며 차량의 바퀴에 장착됩니다. 주행, 코너링 및 제동에 필요한 그립을 제공합니다. 타이어 공기압과 타이어 마모는 차량의 성능과 핸들링에 영향을 미칠 수 있으므로 고려해야 할 두 가지 중요한 요소입니다.

권장 타이어 공기압을 찾을 수 있는 곳:

차량의 운전석 쪽 B-필러에 있는 공기 주입 라벨에서 권장 타이어 공기압 사양을 확인할 수 있습니다. 또는 차량 설명서에서 이 정보를 참조할 수 있습니다. 권장 타이어 공기압은 속도와 탑승자 수 또는 차량의 최대 적재량에 따라 달라질 수 있습니다.

타이어 재공기:

타이어 공기압을 점검할 때는 타이어가 차가울 때 확인하는 것이 중요합니다. 이는 타이어가 주변 온도와 동일한 온도가 되도록 차량을 최소 3시간 동안 그대로 두는 것을 의미합니다.

타이어를 다시 채우려면:

    차량의 권장 타이어 공기압을 확인하십시오.
    공기 펌프에 제공된 지침을 따르고 타이어에 올바른 공기압을 주입하십시오.
    차량의 중앙 화면에서 "차량 상태" 앱을 엽니다.
    "타이어 공기압" 탭으로 이동합니다.
    "압력 보정" 옵션을 누르고 조치를 확인하십시오.
    타이어 공기압을 보정하기 위해 30km/h 이상의 속도로 몇 분 동안 차를 운전하십시오.

참고: 경우에 따라 타이어 공기압과 관련된 경고 기호 또는 메시지를 지우기 위해 15분 이상 운전해야 할 수도 있습니다. 경고가 지속되면 타이어를 식히고 위의 단계를 반복하십시오.

바람 빠진 타이어:

운전 중 타이어 펑크가 났을 경우 일시적으로 펑크를 봉인하고 타이어 이동성 키트를 사용하여 타이어에 다시 공기를 주입할 수 있습니다. 이 키트는 일반적으로 차량의 수하물 보관함 안감 아래에 보관됩니다.

타이어 이동성 키트 사용 지침:

    차량의 테일게이트 또는 트렁크를 엽니다.
    타이어 모빌

In [15]:
body = json.dumps({
  "prompt": f"\n\nHuman:{prompt_data}\n\nAssistant:",
  "max_tokens_to_sample": 500,
  "temperature": 0.3,
  "top_k": 250,
  "top_p": 1,
  "stop_sequences": ["\n\nHuman:"]
})

inference_model(body, modelId, accept, contentType)

prompt_data:
 ## 사이에 제공된 정보만을 기준으로 질문에 답하고 단계별 안내를 제공합니다.
#
타이어 및 타이어 공기압:

타이어는 검은색 고무로 만들어졌으며 차량의 바퀴에 장착됩니다. 주행, 코너링 및 제동에 필요한 그립을 제공합니다. 타이어 공기압과 타이어 마모는 차량의 성능과 핸들링에 영향을 미칠
수 있으므로 고려해야 할 두 가지 중요한 요소입니다.

권장 타이어 공기압을 찾을 수 있는 곳:

차량의 운전석 쪽 B-필러에 있는 공기 주입 라벨에서 권장 타이어 공기압 사양을 확인할 수 있습니다. 또는 차량 설명서에서 이 정보를 참조할 수 있습니다. 권장 타이어 공기압은
속도와 탑승자 수 또는 차량의 최대 적재량에 따라 달라질 수 있습니다.

타이어 재공기:

타이어 공기압을 점검할 때는 타이어가 차가울 때 확인하는 것이 중요합니다. 이는 타이어가 주변 온도와 동일한 온도가 되도록 차량을 최소 3시간 동안 그대로 두는 것을 의미합니다.

타이어를 다시 채우려면:

    차량의 권장 타이어 공기압을 확인하십시오.
    공기 펌프에 제공된 지침을 따르고 타이어에 올바른 공기압을 주입하십시오.
    차량의 중앙 화면에서 "차량 상태" 앱을 엽니다.
    "타이어 공기압" 탭으로 이동합니다.
    "압력 보정" 옵션을 누르고 조치를 확인하십시오.
    타이어 공기압을 보정하기 위해 30km/h 이상의 속도로 몇 분 동안 차를 운전하십시오.

참고: 경우에 따라 타이어 공기압과 관련된 경고 기호 또는 메시지를 지우기 위해 15분 이상 운전해야 할 수도 있습니다. 경고가 지속되면 타이어를 식히고 위의 단계를 반복하십시오.

바람 빠진 타이어:

운전 중 타이어 펑크가 났을 경우 일시적으로 펑크를 봉인하고 타이어 이동성 키트를 사용하여 타이어에 다시 공기를 주입할 수 있습니다. 이 키트는 일반적으로 차량의 수하물 보관함
안감 아래에 보관됩니다.

타이어 이동성 키트 사용 지침:

    차량의 테일게이트 또는 트렁크를 엽니다.
    타이어 모빌리

7. Stream 답변 생성

In [16]:
from IPython.display import display_markdown, Markdown, clear_output
response = boto3_bedrock_cli.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = []
i = 1
if stream:
  for event in stream:
    chunk = event.get('chunk')
    if chunk:
      chunk_obj = json.loads(chunk.get('bytes').decode())
      text = chunk_obj['completion']
      clear_output(wait=True)
      output.append(text)
      display_markdown(Markdown(''.join(output)))
      i+=1


 Audi A8의 펑크 난 타이어를 고치기 위해서는 다음과 같은 단계를 따르세요:

1. 차량의 테일게이트나 트렁크를 엽니다.

2. 러기지 영역의 라이닝을 들어올려 타이어 모빌리티 키트에 접근합니다. 

3. 타이어 이동성 키트와 함께 제공된 지침에 따라 타이어의 펑크를 밀봉합니다.

4. 키트를 사용하여 타이어에 다시 공기를 넣습니다.

5. 사용 후 키트를 원래 위치에 안전하게 다시 집어넣습니다. 

6. 사용한 실란트 병은 폐기하고 새 병으로 교체합니다.

7. 펑크 난 타이어는 가능한 빨리 전문가에게 수리를 맡기거나 교체하는 것이 좋습니다.

8. 타이어 이동성 키트는 임시 해결책이므로 최대 80km/h로 10분 이내에 서비스 센터를 찾아가세요.